In [1]:
pip install googlemaps


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install openpyxl


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import time
import googlemaps
import pandas as pd
import csv
from pprint import pprint
import openpyxl

In [3]:
API_KEY = open('API.txt','r').read()
map_client = googlemaps.Client(API_KEY)

In [4]:
# Load the neighborhood details from a CSV file into a DataFrame
neighborhood_df = pd.read_csv('neighborhood_data.csv')

# Function to fetch gyms in a neighborhood
def fetch_gyms(neighborhood):
    allowed_columns = ["place_id", "geometry"]  # Specifying the allowed columns

    gym_list = []
    response = map_client.places_nearby(
        location=neighborhood["location"],
        radius=neighborhood["radius"],
        type='gym',  
    )

    while 'results' in response:
        for result in response['results']:
            # Filter columns to keep only allowed ones
            filtered_result = {key: result[key] for key in allowed_columns}

            # Add neighborhood information
            filtered_result['neighborhood'] = neighborhood["location"]

            gym_list.append(filtered_result)

        time.sleep(2)  # Pause to avoid rate limiting issues
        if 'next_page_token' in response:
            response = map_client.places_nearby(
                location=neighborhood["location"],
                radius=neighborhood["radius"],
                type='gym',
                page_token=response['next_page_token']
            )
        else:
            break

    return gym_list

# Fetch gyms for each neighborhood in the DataFrame
all_gyms = []

for index, row in neighborhood_df.iterrows():
    neighborhood_info = {
        "name": row["name"],
        "location": row["location"],
        "radius": row["radius"]
    }
    neighborhood_gyms = fetch_gyms(neighborhood_info)
    all_gyms.extend(neighborhood_gyms)

# Create a DataFrame from the combined list of gyms
gym_df = pd.DataFrame(all_gyms)

# Save the merged dataset to an Excel file
gym_df.to_excel('Allowed_gyms.xlsx', index=False)